In [ ]:
import torch
import time
import matplotlib.pyplot as plt
from collections import defaultdict

from sache.train import SAE, SwitchSAE
device='cuda'
n_trials =10
n_experts=16
n_feats=24576
d_in = 768
switch_sae = SwitchSAE(n_features=n_feats, n_experts=n_experts, d_in=d_in, device=device)
sae = SAE(n_features=n_feats, d_in=d_in, device=device)

saes = [sae, switch_sae]
bss = [8192, 4096, 2048, 1024, 512, 256]
results = defaultdict(list)
for bs in bss:
    for s in saes:
        # warm up
        input = torch.randn(bs, d_in, device=device)
        s.forward_descriptive(input)

        torch.cuda.synchronize()
        start = time.time()

        for i in range(n_trials):
            input = torch.randn(bs, d_in, device=device)
            s.forward_descriptive(input)

        torch.cuda.synchronize()
        end = time.time()
        results[str(type(s))].append(end-start)

for k in results.keys():
    plt.plot(bss, results[k], label=k)
    plt.legend()
    plt.xlabel('Batch size')
    plt.ylabel('Time (s)')
    plt.title('Forward descriptive')
